In [2]:
import numpy as np
import sklearn
import os
import random
import math

from tqdm import tqdm
from scipy.io import loadmat

np.random.seed(1)


# Data Processing

In [3]:
full_train_data = loadmat('Data/sarcos_inv.mat')['sarcos_inv']
full_test_data = loadmat('Data/sarcos_inv_test.mat')['sarcos_inv_test']

In [4]:
def getData (full_train_data, full_test_data, num_train, num_validation, num_test):
    
    idx_train = np.random.randint(full_train_data.shape[0], size=num_train)
    idx_validation = np.random.randint(full_train_data.shape[0], size=num_validation)
    idx_test = np.random.randint(full_test_data.shape[0], size=num_test)
    
    train_data = full_train_data[idx_train, : ]
    validation_data = full_train_data[idx_validation, :]
    test_data = full_test_data[idx_test, : ]
    return (train_data, validation_data, test_data)

def Sep_X_and_Y(data, x_dim, y_dim):
    X = data[:, :x_dim]
    Y = data[:, x_dim: x_dim + y_dim ]
    return X, Y

def Unfold_Y(X, Y):
    num_tasks = Y.shape[1]
    X_new = np.vstack([X]*num_tasks)
    Y_new = np.ndarray.flatten(Y, 'F')
    T = np.arange(num_tasks)
    T_new = np.repeat(T, X.shape[0])
    return X_new, Y_new, T_new

def append_one(X):
    n = len(X)
    ones = np.zeros((n, 1)) + 1
    X_new = np.concatenate([ones, X], axis= 1)
    return X_new

In [5]:
(train_data, validation_data, test_data) = getData(full_train_data, full_test_data, 500, 100, 500)

X_train_org, Y_train_org = Sep_X_and_Y(train_data, 21, 2)
X_valid_org, Y_valid_org = Sep_X_and_Y(validation_data, 21, 2)
X_test_org , Y_test_org  = Sep_X_and_Y(test_data, 21, 2)

X_train, Y_train, T_train = Unfold_Y(X_train_org, Y_train_org)
X_valid, Y_valid, T_valid = Unfold_Y(X_valid_org, Y_valid_org)
X_test , Y_test , T_test  = Unfold_Y(X_test_org , Y_test_org )

X_train = append_one(X_train)
X_valid = append_one(X_valid)
X_test = append_one(X_test)

# Multitask Gaussian Process

In [6]:
def Kernel_input(x1, x2):  # K_input part of ICM kernel
    numerator    = 2*np.dot(np.dot(x1, Sigma_u), x2 )
    denominator1 = 1 + 2*np.dot(np.dot(x1, Sigma_u), x1 )
    denominator2 = 1 + 2*np.dot(np.dot(x2, Sigma_u), x2 )
    denominator  = math.sqrt(denominator1*denominator2) # See "Computing with infinite networks" for calculation of Expectation term 
    
    Expectation_term = (2/np.pi) * math.asin( numerator/denominator )
    similarity       = C_term +  Expectation_term  # See "Multitask Neural networks meet Multitask Gaussian Process" Paper for notation of C-term and Expectation term
    return similarity 


def Kernel_task (t1, t2):   # K_task part of the ICM Kernels
    return Omega2[t1, t2]


def Kernel(x1, t1, x2, t2): # ICM Kernel - product of input and task dependent components
    return Kernel_input(x1, x2)*Kernel_task(t1, t2)


def mtgp_fit (X_train, T_train, Y_train, Noise_variance ): # Simple MTGP implementation with specified Kernels (here ICM kernels)
    N = len(X_train)
    K = np.zeros((N, N))
        
    for i in range(0, N):
        for j in range(0, N):
            K[i, j] = Kernel(X_train[i], T_train[i], X_train[j], T_train[j] )
            
    B = np.zeros((N, N))
    for i in range(0, N):
        B[i, i] = Noise_variance[T_train[i]]
         
    
    C = K + B
    C_inv = np.linalg.inv(C)
    alpha = np.dot(C_inv, Y_train)
    model = {
        "X_train" :  X_train,
        "T_train" :  T_train,
        "Y_train" :  Y_train,
        "Noise_variance" : Noise_variance,
        "C_inv"   :  C_inv,
        "alpha"   :  alpha,
    }
    
    return model



def mtgp_predict(X, T, model):
    
    n_train  = len(model["X_train"])
    n        = len(X)
    K        = np.zeros((n_train, n))
    
    for i in range(0, n_train):
        for j in range(0, n):
            K[i, j] = Kernel( model["X_train"][i], model["T_train"][i], X[j] , T[j] )
            
    y_pred = np.dot(K.T, model["alpha"] )
    return y_pred


from sklearn.metrics import mean_squared_error


def get_stats (y_true, y_predict):
    msr = mean_squared_error(y_true, y_predict)
    stats ={
        "msr" : msr
    }
    
    return stats
    

# Hyperparameters

In [12]:
# Temporary variables for convineance - remove after tuning

temp_21_vec = np.zeros(22) + 1
temp_2x2    = np.identity(2)
temp_2      = [1e-5, 1e-5]

# Kernel Hyperparameters - Prior Hyperparameters

Sigma_u =  np.diag( temp_21_vec ) # diagonal matrix of size = dimensions of X (current example: 21)
C_term  =  1             # Proportionality Constant term for bias variance 
Omega2  =  temp_2x2      # Covariance between tasks - Symmetric matrix of size T X T 


# Noise Variance - Likelihood Hyperparameters
Noise_variance = temp_2 # Noise variance - Vector of size T, number of tasks 


# Bayesian Optimization

In [18]:
def get_Negative_Loss (a):
    global Noise_variance
    global Sigma_u
    Sigma_u = math.pow(10, a)*np.diag( temp_21_vec )
#     Noise_variance = np.array([math.pow(10, Noise_variance1), math.pow(10, Noise_variance2) ])
    model = mtgp_fit (X_train, T_train, Y_train, Noise_variance )
    predict = mtgp_predict(X_valid, T_valid, model)
    stats = get_stats(Y_valid, predict)
    return -stats['msr']


In [19]:
from bayes_opt import BayesianOptimization


pbounds = {
            "a" : (-10, 2)
          }


optimizer = BayesianOptimization(
    f= get_Negative_Loss,
    pbounds=pbounds,
    random_state=1,
)



In [20]:
optimizer.maximize(
    init_points=2,
    n_iter=10,
)

|   iter    |  target   |     a     |
-------------------------------------
|  1        | -24.75    | -4.996    |
|  2        | -80.09    | -1.356    |
|  3        | -24.75    | -4.996    |
|  4        | -24.59    | -4.961    |
|  5        | -24.45    | -4.933    |
|  6        | -23.68    | -4.787    |
|  7        | -22.76    | -4.62     |
|  8        | -21.58    | -4.427    |
|  9        | -20.52    | -4.243    |
|  10       | -22.99    | -4.068    |
|  11       | -20.62    | -4.286    |
|  12       | -20.53    | -4.253    |


# Single Layer Neural Network

In [29]:
from sklearn.neural_network import MLPRegressor

num_hidd_units = 1000

for rs in range(0, 1):
    clf = MLPRegressor(hidden_layer_sizes= (num_hidd_units, ), activation='logistic' ,random_state=rs,  solver='adam', max_iter=100000)
    clf.fit(X_train_org, Y_train_org)
    Y_pred_NN = clf.predict(X_valid_org)
    Y_pred_NN1 = Y_pred_NN.flatten()
    Y_valid_org1 = Y_valid_org.flatten()
    
    stats = get_stats(Y_pred_NN1, Y_valid_org1)
    print( -stats['msr'] )

-23.77582466812858
